In [ ]:
import requests
import uuid
import time
import json

with open('./data/api_url.txt','r') as api_url : 
    api_url = api_url.readline()

with open('./data/secret_key.txt','r') as secret_key : 
    secret_key = secret_key.readline()

image_file = './data/20211019_172952.png'
output_file = './output.json'

request_json = {
    'images': [
        {
            'format': 'png',
            'name': 'demo',
            'templateIds': [11112]
        }
    ],
    'requestId': str(uuid.uuid4()),
    'version': 'V2',
    'timestamp': int(round(time.time() * 1000))
}

payload = {'message': json.dumps(request_json).encode('UTF-8')}
files = [
  ('file', open(image_file,'rb'))
]
headers = {
  'X-OCR-SECRET': secret_key
}

response = requests.request("POST", api_url, headers=headers, data = payload, files = files)

res = json.loads(response.text.encode('utf8'))
print(res)

with open(output_file, 'w', encoding='utf-8') as outfile:
    json.dump(res, outfile, indent=4, ensure_ascii=False)

In [11]:
# 날짜
date = res['images'][0]['fields'][0]['inferText'].split('\n')

# 시간
time = res['images'][0]['fields'][1]['inferText'].split('\n')

# 수집동의
checkbox = res['images'][0]['fields'][2]['inferText'].split('\n')

# 시군구
sigungu = res['images'][0]['fields'][3]['inferText'].split('\n')

# 전화번호
tel = res['images'][0]['fields'][3]['inferText'].split('\n')

# 비고
extra = res['images'][0]['fields'][4]['inferText'].split('\n')

In [19]:
# max length 
length = 0 
li = [date, time, checkbox, sigungu, tel, extra]
for i in li : 
    length = max(len(i), length)


# 패딩 맞춰주기 
import numpy as np 
df_li = []
for i in li :
     df_li.append(np.pad(i, (0,length - len(i)), 'constant', constant_values=0))

In [ ]:
import pandas as pd
df = pd.DataFrame(df_li).transpose()
df.columns = ['date', 'time', 'checkbox', 'sigungu', 'tel', 'extra']
df